<h1>Grafo delle co-authorship tra gli articoli del corso di informatica</h1>
<p>Dal dataset degli articoli di informatica vengono filtrati solamente quelli dei professori che attualmente sono nella lista del suddetto dipartimento.</p>

In [1]:
from matplotlib.font_manager import font_scalings
from pyvis.network import Network
import networkx as nx
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

<h2>Caricamento dei dati</h2>

<h3>Dataset degli articoli di informatica</h3>
<p>In formato csv e convertito a dataframe</p>

In [3]:
# Carico il dataset
df_informatica = pd.read_csv("../data/informatica.csv")
df_informatica

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,ISBN,CODEN,PubMed ID,Language of Original Document,Abbreviated Source Title,Document Type,Publication Stage,Open Access,Source,EID
0,"Mari, M.; Snidaro, L.","Mari, Marco (59388337700); Snidaro, Lauro (650...",59388337700; 6507499895,Survey of Neural Network Approaches to Target ...,2026,Information Fusion,127,NaN,103789,NaN,...,NaN,NaN,NaN,English,Inf. Fusion,Article,Final,All Open Access; Hybrid Gold Open Access,Scopus,2-s2.0-105017548985
1,"Mari, M.; Snidaro, L.","Mari, Marco (59388337700); Snidaro, Lauro (650...",59388337700; 6507499895,Ensemble of KalmanNets with innovation-based a...,2026,Information Fusion,127,NaN,103777,NaN,...,NaN,NaN,NaN,English,Inf. Fusion,Article,Final,NaN,Scopus,2-s2.0-105017546340
2,"Ceschia, S.; Di Gaspero, L.; Rosati, R.M.; Sch...","Ceschia, Sara (35193496000); Di Gaspero, Luca ...",35193496000; 6505944235; 57406536600; 6701629145,Multi-neighborhood simulated annealing for the...,2026,International Transactions in Operational Rese...,33,1,NaN,38,...,NaN,NaN,NaN,English,Int. Trans. Oper. Res.,Article,Final,All Open Access; Hybrid Gold Open Access,Scopus,2-s2.0-85211480324
3,"Pagliarini, R.","Pagliarini, Roberto (25229039800)",25229039800,Differential Flux-Balance Analysis Infers Meta...,2026,Lecture Notes in Computer Science,16051 LNCS,NaN,NaN,155,...,9789819698936; 9789819698042; 9789819698110; 9...,NaN,NaN,English,Lect. Notes Comput. Sci.,Conference paper,Final,NaN,Scopus,2-s2.0-105022894987
4,"Madni, H.A.; Shujat, H.; de Nardin, A.; Zottin...","Madni, Hussain Ahmad (57195220347); Shujat, Ha...",57195220347; 59705771800; 57259162600; 5795942...,FsBAD: Data-efficient feature reconstruction f...,2026,Pattern Recognition Letters,199,NaN,NaN,113,...,NaN,NaN,NaN,English,Pattern Recogn. Lett.,Article,Final,All Open Access; Hybrid Gold Open Access,Scopus,2-s2.0-105021263351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4438,"Beŕtoni, A.; Mauri, G.; Sabadini, N.","Beŕtoni, Alberto (7005822404); Mauri, Giancarl...",7005822404; 7004284991; 6602383233,Equivalence and membership problems for regula...,1982,Lecture Notes in Computer Science,140 LNCS,NaN,NaN,61,...,9789819698936; 9789819698042; 9789819698110; 9...,NaN,NaN,English,Lect. Notes Comput. Sci.,Conference paper,Final,NaN,Scopus,2-s2.0-85037042961
4439,"Berry, D.M.; Ghezzi, C.; Mandrioli, D.; Tisato...","Berry, Daniel M. (7402256761); Ghezzi, Carlo (...",7402256761; 16512874900; 6603733719; 59777078600,Language constructs for real-time distributed ...,1982,Computer Languages,7,1,NaN,11,...,NaN,COLAD,NaN,English,Comput Lang,Article,Final,NaN,Scopus,2-s2.0-0020003724
4440,"Beŕtoni, A.; Mauri, G.; Sabadlni, N.","Beŕtoni, Alberto (7005822404); Mauri, Giancarl...",7005822404; 7004284991; 57203385151,A characterization of the class of functions c...,1981,Proceedings of the Annual ACM Symposium on The...,NaN,NaN,NaN,168,...,9798400715105; 0897910176; 9781605588179; 9781...,NaN,NaN,English,Proc. Annu. ACM Symp. Theory Comput.,Conference paper,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85049612709
4441,"Mauri, G.; Sabadini, N.","Mauri, Giancarlo (7004284991); Sabadini, Nicol...",7004284991; 6602383233,A # P-complete problem over arithmetical trees,1981,Lecture Notes in Computer Science,112 LNCS,NaN,NaN,308,...,9789819698936; 9789819698042; 9789819698110; 9...,NaN,NaN,English,Lect. Notes Comput. Sci.,Conference paper,Final,NaN,Scopus,2-s2.0-85034452402


<h3>Creazione della lista dei professori di informatica</h3>
<p>I nomi sono stati copiati dal sito dell'università e incollati in un file di testo.</p>

In [3]:
# Carico la lista dei professori dal file txt
with open("../data/professori_informatica.txt", "r", encoding="utf-8") as f:
    professori_informatica = [riga.strip() for riga in f]

def normalize_prof(name):
    cognome, nome = [x.strip() for x in name.split(',')]
    return f"{cognome} {nome}"

professori_informatica = set(normalize_prof(n) for n in professori_informatica)
professori_informatica = list(professori_informatica)
professori_informatica.sort()
print(professori_informatica)

['Bellettini Giovanni', 'Bozzo Enrico', 'Brajnik Giorgio', 'Burigat Stefano', 'Buttussi Fabio', 'Chittaro Luca', 'Cimatti Alessandro', 'Comini Marco', 'Coppola Paolo', 'Dattolo Antonina', 'Della Mea Vincenzo', 'Della Monica Dario', 'Di Gianantonio Pietro', 'Dimonte Vincenzo', 'Dovier Agostino', 'Drioli Carlo', 'D’Agostino Giovanna', 'Fogolari Federico', 'Fontana Federico', 'Formisano Andrea', 'Fortuna Sara', 'Franceschet Massimo', 'Fusiello Andrea', 'Geatti Luca', 'Lancia Giuseppe', 'Lenisa Marina', 'Liessi Davide', 'Maddalena Eddy', 'Mameli Valentina', 'Micheloni Christian', 'Miculan Marino', 'Mirolo Claudio', 'Mizzaro Stefano', 'Montanari Angelo', 'Omero Paolo', 'Pace Luigi', 'Piazza Carla', 'Policriti Alberto', 'Puppis Gabriele', 'Riccio Vincenzo', 'Ritacco Ettore', 'Roitero Kevin', 'Romani Giulio', 'Scagnetto Ivan', 'Serra Giuseppe', 'Vermiglio Rossana', 'Vidoni Paolo']


<h2>Funzione per estrarre gli autori dalla relativa colonna del dataframe</h2>
<p>Gli autori sono memorizzati come: "Cognome, Nome (ID); ..."<br>
Quindi vogliamo trasformarli in "Cognome Nome"
</p>

In [4]:
def pulisci_autori(author_string):
    if pd.isna(author_string):
        return []
    
    autori = []
    for a in author_string.split(";"):
        a = a.strip()
        # prendo la parte prima di "("
        a = a.split("(")[0].strip()
        
        # separo Cognome, Nome
        if "," in a:
            cognome, nome = a.split(",", 1)
            autori.append(f"{cognome.strip()} {nome.strip()}")
        else:
            autori.append(a)
    return autori


<h2>Creazione del grafo</h2>
<p>Viene anche salvato in formato pickle in modo tale che possa essere utilizzato in altri notebook</p>

In [5]:
G = nx.Graph()

for autori_string in df_informatica["Author full names"]:
    lista_autori = pulisci_autori(autori_string)

    # Filtra solo i professori del dipartimento
    autori_prof = [a for a in lista_autori if a in professori_informatica]

    # Aggiungi edge tra tutte le coppie di professori che hanno co-firmato
    for i in range(len(autori_prof)):
        for j in range(i + 1, len(autori_prof)):
            a1, a2 = autori_prof[i], autori_prof[j]

            if G.has_edge(a1, a2):
                G[a1][a2]["weight"] += 1
            else:
                G.add_edge(a1, a2, weight=1)
            
with open("./grafi_pickle/G_inf.pkl", "wb") as f:
    pickle.dump(G, f)

<h3>Dati del grafo</h3>
<ul>
    <li>Numero di nodi (professori coinvolti)</li>
    <li>Numero di archi (co-authorship)</li>
    <li>Per ogni professore i relativi collaboratori</li>
</ul>

In [6]:
print("Numero di nodi (professori coinvolti):", G.number_of_nodes())
print("Numero di archi (co-autorship):", G.number_of_edges())

# Mostra i vicini di un professore, se vuoi
for n in G.nodes():
    print(n, "co-autore con:", list(G.neighbors(n)))


Numero di nodi (professori coinvolti): 36
Numero di archi (co-autorship): 66
Roitero Kevin co-autore con: ['Maddalena Eddy', 'Mizzaro Stefano', 'Coppola Paolo', 'Della Mea Vincenzo', 'Serra Giuseppe']
Maddalena Eddy co-autore con: ['Roitero Kevin', 'Mizzaro Stefano', 'Della Mea Vincenzo']
Mizzaro Stefano co-autore con: ['Roitero Kevin', 'Maddalena Eddy', 'Coppola Paolo', 'Della Mea Vincenzo', 'Serra Giuseppe', 'Scagnetto Ivan', 'Fusiello Andrea', 'Brajnik Giorgio']
Coppola Paolo co-autore con: ['Della Mea Vincenzo', 'Mizzaro Stefano', 'Roitero Kevin', 'Scagnetto Ivan', 'Chittaro Luca']
Della Mea Vincenzo co-autore con: ['Coppola Paolo', 'Mizzaro Stefano', 'Roitero Kevin', 'Serra Giuseppe', 'Maddalena Eddy', 'Scagnetto Ivan']
Lenisa Marina co-autore con: ['Scagnetto Ivan', 'Di Gianantonio Pietro', 'Miculan Marino']
Scagnetto Ivan co-autore con: ['Lenisa Marina', 'Drioli Carlo', 'Di Gianantonio Pietro', 'Mizzaro Stefano', 'Coppola Paolo', 'Della Mea Vincenzo', 'Miculan Marino', 'Chittaro

<h3>Visualizzazione del grafo tramite libreria Pyvis</h3>
<p>Il file viene salvato in formato .html nella relativa cartella</p>

In [10]:

# Crea un grafo pyvis
net = Network(height="750px", width="100%", bgcolor="black", font_color="white", notebook=True, cdn_resources="remote", filter_menu=True)

# Forza fisica per un layout più bello
net.force_atlas_2based()

# Aggiungi nodi e archi da NetworkX a PyVis
for node in G.nodes():
    net.add_node(
        node,
        label=node,
        title=node,
        color="#4A90E2",   # colore nodi
        shape="dot",
        size=15
    )

for u, v, data in G.edges(data=True):
    weight = data.get("weight", 1)
    net.add_edge(
        u, 
        v, 
        value=weight,
        title=f"Co-authored papers: {weight}"
    )

# Salva in HTML

output_path = "../html/grafi_co-authorship/grafo_coauthorship_informatica.html"
net.write_html(output_path)
print("File generato:", output_path)
net.show(output_path)

File generato: ../html/grafi_co-authorship/grafo_coauthorship_informatica.html
../html/grafi_co-authorship/grafo_coauthorship_informatica.html
